# Knowledge Distillation in MHIST Dataset

In [ ]:
import os
import sys
import csv
import pandas as pd
import tensorflow as tf
import numpy as np
import math
import random

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Utilities import *

In [ ]:
# Prepare data
path = "mhist_dataset/images"
CSVfile = "mhist_dataset/annotations.csv"

data, file_names = load_mhist_images(path)
labels = pd.read_csv(CSVfile, usecols = [1])
Partitions = pd.read_csv(CSVfile, usecols = [3])
labels = labels.to_numpy()
Partitions = Partitions.to_numpy()
X_train = []
X_test = []
y_train = []
y_test = []
for i in range(len(data)):
    if Partitions[i] == 'train':
        X_train.append(data[i])
        if (labels[i] == 'SSA'):
            y_train.append([1,0])
        if (labels[i] == 'HP'):
            y_train.append([0,1])
    if Partitions[i] == 'test':
        X_test.append(data[i])
        if (labels[i] == 'SSA'):
            y_test.append([1,0])
        if (labels[i] == 'HP'):
            y_test.append([0,1])

c = list(zip(X_train, y_train))
random.shuffle(c)
X_train, y_train = zip(*c)

c = list(zip(X_test, y_test))
random.shuffle(c)
X_test, y_test = zip(*c)

#RotatedX_test = np.rot90(X_test,axes=(-2,-1))
#X_test.append(data[RotatedX_test])
#for i in range(X_test.shape[0]))
    #y_test.append(y_test[i])

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

n_batches = 32
Train_Data = []
Train_Label = []
for i in range(math.ceil(X_train.shape[0]/n_batches)):
    # Local batches and labels
    local_X, local_y = X_train[i*n_batches:(i+1)*n_batches,], y_train[i*n_batches:(i+1)*n_batches,]
    Train_Data.append(local_X)
    Train_Label.append(local_y)

Test_Data = []
Test_Label = []
for i in range(math.ceil(X_test.shape[0]/n_batches)):
    # Local batches and labels
    local_X, local_y = X_test[i*n_batches:(i+1)*n_batches,], y_test[i*n_batches:(i+1)*n_batches,]
    Test_Data.append(local_X)
    Test_Label.append(local_y)

print(Train_Data[0].shape)
print(Train_Data[-1].shape)
print(Test_Data[0].shape)
print(Test_Data[-1].shape)

In [ ]:
# Load Resnet
resNetBase= tf.keras.applications.resnet_v2.ResNet50V2(
    include_top = False,
    weights='imagenet',
    input_shape=(224,224,3),
    pooling=None,
)
for layer in resNetBase.layers[:-1]:
    layer.trainable = False
x = tf.keras.layers.Flatten()(resNetBase.output)
x = tf.keras.layers.Dense(2)(x)
restNet = tf.keras.Model(inputs=resNetBase.input, outputs=x)

In [ ]:
train_evaluate_resnet(restNet,Train_Data,Test_Data,Train_Label,Test_Label)

In [ ]:
studenModel2 = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top = False,
    weights='imagenet',
    input_shape=(224,224,3),
    pooling=None,
)
for layer in studenModel2.layers[:-1]:
    layer.trainable = False
x = tf.keras.layers.Flatten()(studenModel2.output)
x = tf.keras.layers.Dense(2)(x)
mobileNet = tf.keras.Model(inputs=studenModel2.input, outputs=x)


In [ ]:
train_and_evaluate_mobileNet_using_KD(mobileNet,restNet,Train_Data,Test_Data,Train_Label,Test_Label, 0.5,4)

In [ ]:
teachingAssistantAcc = testTransferedModel(restNet,Test_Data,Test_Label)
teachingAssistantAcc = testTransferedModel(mobileNet,Test_Data,Test_Label)

In [ ]:
testACC = []
tempratureValues = [1, 2, 4, 16, 32, 64]
for temp in tempratureValues:
    st = getStudentModel()
    train_and_evaluate_mobileNet_using_KD(mobileNet,restNet,Train_Data,Test_Data,Train_Label,Test_Label, 0.5,4)
    testACC += [testTransferedModel(mobileNet,Test_Data,Test_Label)]
print(testACC)